## Initial set-up

In [ ]:
pip install duckdb

In [ ]:
pip install pandas 

In [ ]:
pip install ipython-sql

In [ ]:
pip install SQLAlchemy

In [ ]:
pip install duckdb-engine

In [ ]:
import duckdb
import pandas as pd
import sqlalchemy
# No need to import duckdb_engine
#  SQLAlchemy will auto-detect the driver needed based on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Instantiate duckdb in-memory

In [ ]:
%sql duckdb:///:memory:

## Install JSON extension

Since the FHIR resources are stored as JSON in a column and we want to query parts of the JSON we install the JSON extension.

In [ ]:
%%sql

install 'json';
load 'json';

DuckDB can directly query parquet files with full support for SQL. No import of data necessary !

In [ ]:
%%sql
select count(*) from 'resource-snappy.parquet';

In [ ]:
%%sql
select * from 'resource-snappy.parquet' using sample 100;

For convinience lets create a resource table and then import the parquet file into this table and then use the table to run SQL queries.

In [ ]:
%%sql
create table resource(id VARCHAR, resourceType VARCHAR, resourceJson JSON);

In [ ]:
%%sql
INSERT INTO resource SELECT * FROM read_parquet('resource-snappy.parquet');

In [ ]:
%%sql
select count(*) from resource where json_extract_string(resourceJson, '$.resourceType') = 'Observation' and json_extract_string(resourceJson, '$.category[0].coding[0].code') = 'laboratory';

In [ ]:
%%sql
select unnest(from_json(json_extract(resourceJson, '$.code.coding'), '[{"system":"VARCHAR", "code":"VARCHAR", "display":"VARCHAR"}]')) from resource using sample 100;

In [ ]:
%%sql
select list_contains(list_transform(from_json(resourceJson ->> '$.code.coding', '["json"]'), x -> json_extract_string(json(x), 'code')), '8302-2') from resource where json_extract_string(resourceJson, '$.resourceType') = 'Observation' and json_extract_string(resourceJson, '$.category[0].coding[0].code') = 'laboratory';

In [ ]:
%%sql
select resourceJson  from resource where list_contains(list_transform(from_json(resourceJson ->> '$.code.coding', '["json"]'), x -> json_extract_string(json(x), 'code')), '8302-2');

### SQL on FHIR using Parquet files

In [ ]:
%%sql
SELECT count(*) FROM read_parquet('./fhir_parquet/encounter/*.parquet');

In [ ]:
%%sql
create table patient as SELECT * FROM read_parquet('./fhir_parquet/patient/*.parquet');

In [ ]:
%%sql
create table observation as SELECT * FROM read_parquet('./fhir_parquet/observation/*.parquet');

In [ ]:
%%sql
create table condition as SELECT * FROM read_parquet('./fhir_parquet/condition/*.parquet');

In [ ]:
%%sql
WITH coding as (SELECT UNNEST(observation.code.coding) c from observation)
SELECT subject.reference,
    AVG(value.quantity.value) avg_hdl
FROM observation o, coding
WHERE c.system = 'http://loinc.org' AND
    c.code = '2085-9' AND
      o.effective.datetime > '2017'
GROUP BY subject.reference;

In [ ]:
%%sql
WITH coding as (SELECT UNNEST(observation.code.coding) c from observation),
vscte as (SELECT UNNEST(condition.verificationstatus.coding) vs from condition), 

-- Create a view of patients with a diabetes condition (prediabetes or diabetes)
diabetics as (
  SELECT DISTINCT subject.reference person_ref
  FROM condition, coding, vscte
  WHERE vs.code = 'confirmed' AND
        c.system = 'http://snomed.info/sct' AND
        (c.code = '15777000' OR 
         c.code = '44054006') 
),

-- Create a view of all HbA1c values.
hba1c_values as (
  SELECT subject.reference person_ref,
         value.quantity.value,
         value.quantity.unit,
         c.system,
         c.code,
         c.display,    
         o.effective.dateTime as effectivedatetime
  FROM observation o, coding
  WHERE c.system = 'http://loinc.org' AND
        c.code = '4548-4' AND
        status = 'final'

)

-- Use the above views to find diabetics who have had a
-- high HbA1c value since the start of say 2017
SELECT d.person_ref,
       h.value,
       h.display,
       h.effectivedatetime
FROM diabetics d
JOIN hba1c_values h ON d.person_ref = h.person_ref
WHERE h.value > 6.5 AND
      h.effectivedatetime > '2017';